# Use Model Context Protocol (MCP) as tools with Strands Agent

## Overview
The [Model Context Protocol (MCP)](https://modelcontextprotocol.io/introduction) is an open protocol that standardizes how applications provide context to Large Language Models (LLMs). Strands AI SDK integrates with MCP to extend agent capabilities through external tools and services.

MCP enables communication between agents and MCP servers that provide additional tools. The Strands Agent SDK includes built-in support for connecting to MCP servers and using their tools.

In this example we will show you how to use MCP tools on your Strands Agent. We will use the [AWS Documentation MCP server](https://awslabs.github.io/mcp/servers/aws-documentation-mcp-server/) which provides tools to access AWS documentation, search for content, and get recommendations. This MCP server has 3 main features:

- **Read Documentation**: Fetch and convert AWS documentation pages to markdown format
- **Search Documentation**: Search AWS documentation using the official search API
- **Recommendations**: Get content recommendations for AWS documentation pages



## Agent Details
<div style="float: left; margin-right: 20px;">
    
|Feature             |Description                                        |
|--------------------|---------------------------------------------------|
|Feature used        |MCP Tools                                          |
|Agent Structure     |Single agent architecture                          |

</div>

## Architecture

<div style="text-align:center">
    <img src="images/architecture.png" width="65%" />
</div>

## Key Features
* **Single agent architecture**: this example creates a single agent that interacts with MCP tools
* **MCP tools**: Integration of MCP tools with your agent

## Setup and prerequisites

### Prerequisites
* Python 3.10+
* AWS account
* Anthropic Claude 3.7 enabled on Amazon Bedrock

Let's now install the requirement packages for our Strands Agent agent

In [1]:
# installing pre-requisites
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.9/17.9 MB 17.8 MB/s  0:00:01m0:00:0100:01


### Importing dependency packages

Now let's import the dependency packages

In [2]:
import threading
import time
from datetime import timedelta

from mcp import StdioServerParameters, stdio_client
from mcp.client.streamable_http import streamablehttp_client
from mcp.server import FastMCP
from strands import Agent
from strands.tools.mcp import MCPClient

### Connect to MCP server using stdio transport

[Transports](https://modelcontextprotocol.io/specification/2025-03-26/basic/transports) in MCP provide the foundations for communication between clients and servers. It handles the underlying mechanics of how messages are sent and received. At the moment there are three standards transport implementations built-in in MCP:

- **Standard Input/Output (stdio)**: enables communication through standard input and output streams. It is particularly useful for local integrations and command-line tools
- **Streamable HTTP**: this replaces the HTTP+SSE transport from previous protocol version. In the Streamable HTTP transport, the server operates as an independent process that can handle multiple client connections. This transport uses HTTP POST and GET requests. Server can optionally make use of Server-Sent Events (SSE) to stream multiple server messages. This permits basic MCP servers, as well as more feature-rich servers supporting streaming and server-to-client notifications and requests.
- **SSE**: legacy transport for HTTP-based MCP servers that use Server-Sent Events transport  

Overall, you should use stdio for building command-line tools, implementing local integrations and working with shell scripts. You should use Streamable HTTP transports when you need a flexible and efficient way for AI agents to communicate with tools and services, especially when dealing with stateless communication or when minimizing resource usage is crucial.

You can also use **custom transports** implementation for your specific needs. 


Let's now connect to the MCP server using stdio transport. First of all, we will use the class `MCPClient` to connect to the [AWS Documentation MCP Server](https://awslabs.github.io/mcp/servers/aws-documentation-mcp-server/). This server provides tools to access AWS documentation, search for content, and get recommendations.

In [3]:
# Connect to an MCP server using stdio transport
stdio_mcp_client = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(
            command="uvx", args=["awslabs.aws-documentation-mcp-server@latest"]
        )
    )
)

#### Setup agent configuration and invoke it

Next we will set our agent configuration using the tools from the `stdio_mcp_client` object we just created. To do so, we need to list the tools available in the MCP server. We can use the `list_tools_sync` method for it. 

After that, we will ask a question to our agent.

In [4]:
# Create an agent with MCP tools
with stdio_mcp_client:
    # Get the tools from the MCP server
    tools = stdio_mcp_client.list_tools_sync()

    # Create an agent with these tools
    agent = Agent(
        model="global.anthropic.claude-sonnet-4-20250514-v1:0",
        tools=tools)

    response = agent("What is Amazon Bedrock pricing model. Be concise.")

I'll search for Amazon Bedrock pricing information to provide you with a concise overview.
Tool #1: search_documentation

Tool #2: read_documentation
## Amazon Bedrock Pricing Model

Amazon Bedrock uses a **pay-per-use pricing model** based on:

**Primary Pricing Components:**
- **Input tokens** - Text you send to the model
- **Output tokens** - Text the model generates back
- **Token volume** - You pay based on the number of tokens processed

**Pricing Options:**
1. **On-Demand** - Pay per token with no upfront costs
2. **Provisioned Throughput** - Purchase dedicated capacity for consistent performance and potentially lower per-token costs

**Additional Charges:**
- **Guardrails** - Extra fees apply when using Amazon Bedrock Guardrails for content filtering
- **Model fine-tuning** - Separate charges for custom model training (if applicable)

Each foundation model has different per-token pricing, which varies by model provider and complexity. You only pay for what you use with no minim

### Connect to MCP server using Streamable HTTP

Let's now connect to the MCP server using Streamable HTTP transport. First let's start a simple MCP server using Streamable HTTP transport. 

For this example we will create our own MCP server. The architecture will look as following

<div style="text-align:center">
    <img src="images/architecture_2.png" width="65%" />
</div>

In [5]:
# Create an MCP server
mcp = FastMCP("Calculator Server")

# Define a tool


@mcp.tool(description="Calculator tool which performs calculations")
def calculator(x: int, y: int) -> int:
    return x + y


@mcp.tool(description="This is a long running tool")
def long_running_tool(name: str) -> str:
    time.sleep(25)
    return f"Hello {name}"


def main():
    mcp.run(transport="streamable-http", mount_path="mcp")

Let's now start a thread with the `streamable-http` server

In [ ]:
thread = threading.Thread(target=main)
thread.start()

INFO:     Started server process [76557]
INFO:     Waiting for application startup.


[09/16/25 14:21:30] INFO     StreamableHTTP session manager started                  ]8;id=414461;file:///Users/sbattoo/Src/AIActivationDay/samples/.venv/lib/python3.13/site-packages/mcp/server/streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=243452;file:///Users/sbattoo/Src/AIActivationDay/samples/.venv/lib/python3.13/site-packages/mcp/server/streamable_http_manager.py#110\110]8;;\

INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:52843 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:52846 - "POST /mcp HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:52847 - "GET /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:52850 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:52862 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:52869 - "DELETE /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:53344 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:53347 - "POST /mcp HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:53348 - "GET /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:53350 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:53352 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:53354 - "DELETE /mcp HTTP/1.1" 200 OK


#### Integrating Streamable HTTP client with Agent

Now let's use `streamablehttp_client` integrate this server with a simple agent. 

In [7]:
def create_streamable_http_transport():
    return streamablehttp_client("http://localhost:8000/mcp")


streamable_http_mcp_client = MCPClient(create_streamable_http_transport)

#### Setup agent configuration and invoke it

Next we will set our agent configuration using the tools from the `streamable_http_mcp_client` object we just created. To do so, we need to list the tools available in the MCP server. We can use the `list_tools_sync` method for it. 

After that, we will ask a question to our agent.

In [8]:
with streamable_http_mcp_client:
    tools = streamable_http_mcp_client.list_tools_sync()

    agent = Agent(
        model="global.anthropic.claude-sonnet-4-20250514-v1:0",
        tools=tools)

    response = str(agent("What is 2 + 2?"))

[09/16/25 14:22:00] INFO     Created new transport with session ID:                  ]8;id=612743;file:///Users/sbattoo/Src/AIActivationDay/samples/.venv/lib/python3.13/site-packages/mcp/server/streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=7434;file:///Users/sbattoo/Src/AIActivationDay/samples/.venv/lib/python3.13/site-packages/mcp/server/streamable_http_manager.py#233\233]8;;\
                             93dce49f14f14a15bc8fa0452252ed16                                                      

                    INFO     HTTP Request: POST http://localhost:8000/mcp "HTTP/1.1 200 OK"         ]8;id=757653;file:///Users/sbattoo/Src/AIActivationDay/samples/.venv/lib/python3.13/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=313973;file:///Users/sbattoo/Src/AIActivationDay/samples/.venv/lib/python3.13/site-packages/httpx/_client.py#1740\1740]8;;\

                    INFO     Received session ID: 93dce49f14f14a15bc8fa0452252ed16           ]8;id=384012;file:///Users/sbattoo/Src/AIActivationDay/samples/.venv/lib/python3.13/site-packages/mcp/client/streamable_http.py\streamable_http.py]8;;\:]8;id=620134;file:///Users/sbattoo/Src/AIActivationDay/samples/.venv/lib/python3.13/site-packages/mcp/client/streamable_http.py#134\134]8;;\

                    INFO     Negotiated protocol version: 2025-06-18                         ]8;id=13656;file:///Users/sbattoo/Src/AIActivationDay/samples/.venv/lib/python3.13/site-packages/mcp/client/streamable_http.py\streamable_http.py]8;;\:]8;id=379193;file:///Users/sbattoo/Src/AIActivationDay/samples/.venv/lib/python3.13/site-packages/mcp/client/streamable_http.py#146\146]8;;\

                    INFO     HTTP Request: POST http://localhost:8000/mcp "HTTP/1.1 202 Accepted"   ]8;id=360227;file:///Users/sbattoo/Src/AIActivationDay/samples/.venv/lib/python3.13/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=894444;file:///Users/sbattoo/Src/AIActivationDay/samples/.venv/lib/python3.13/site-packages/httpx/_client.py#1740\1740]8;;\

...

[09/16/25 14:22:02] INFO     HTTP Request: POST http://localhost:8000/mcp "HTTP/1.1 200 OK"         ]8;id=911321;file:///Users/sbattoo/Src/AIActivationDay/samples/.venv/lib/python3.13/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=253463;file:///Users/sbattoo/Src/AIActivationDay/samples/.venv/lib/python3.13/site-packages/httpx/_client.py#1740\1740]8;;\

                    INFO     Processing request of type CallToolRequest                               ]8;id=400394;file:///Users/sbattoo/Src/AIActivationDay/samples/.venv/lib/python3.13/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=895553;file:///Users/sbattoo/Src/AIActivationDay/samples/.venv/lib/python3.13/site-packages/mcp/server/lowlevel/server.py#623\623]8;;\

The answer is 4.

[09/16/25 14:22:04] INFO     Terminating session: 93dce49f14f14a15bc8fa0452252ed16           ]8;id=785460;file:///Users/sbattoo/Src/AIActivationDay/samples/.venv/lib/python3.13/site-packages/mcp/server/streamable_http.py\streamable_http.py]8;;\:]8;id=202677;file:///Users/sbattoo/Src/AIActivationDay/samples/.venv/lib/python3.13/site-packages/mcp/server/streamable_http.py#630\630]8;;\

                    INFO     HTTP Request: DELETE http://localhost:8000/mcp "HTTP/1.1 200 OK"       ]8;id=891056;file:///Users/sbattoo/Src/AIActivationDay/samples/.venv/lib/python3.13/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=609877;file:///Users/sbattoo/Src/AIActivationDay/samples/.venv/lib/python3.13/site-packages/httpx/_client.py#1740\1740]8;;\

### Direct Tool Invocation

While tools are typically invoked by the agent based on user requests, you can also call MCP tools directly. This can be useful for workflow scenarios where you orchestrate multiple tools together.

In [9]:
query = {"x": 10, "y": 20}

with streamable_http_mcp_client:
    # direct tool invocation
    result = streamable_http_mcp_client.call_tool_sync(
        tool_use_id="tool-123", name="calculator", arguments=query
    )

    # Process the result
    print(f"Calculation result: {result['content'][0]['text']}")

[09/16/25 14:23:59] INFO     Created new transport with session ID:                  ]8;id=163983;file:///Users/sbattoo/Src/AIActivationDay/samples/.venv/lib/python3.13/site-packages/mcp/server/streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=802157;file:///Users/sbattoo/Src/AIActivationDay/samples/.venv/lib/python3.13/site-packages/mcp/server/streamable_http_manager.py#233\233]8;;\
                             7e8a1c1874f24d72b2c6ba57ba9dc570                                                      

                    INFO     HTTP Request: POST http://localhost:8000/mcp "HTTP/1.1 200 OK"         ]8;id=958927;file:///Users/sbattoo/Src/AIActivationDay/samples/.venv/lib/python3.13/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=317574;file:///Users/sbattoo/Src/AIActivationDay/samples/.venv/lib/python3.13/site-packages/httpx/_client.py#1740\1740]8;;\

                    INFO     Received session ID: 7e8a1c1874f24d72b2c6ba57ba9dc570           ]8;id=779038;file:///Users/sbattoo/Src/AIActivationDay/samples/.venv/lib/python3.13/site-packages/mcp/client/streamable_http.py\streamable_http.py]8;;\:]8;id=955734;file:///Users/sbattoo/Src/AIActivationDay/samples/.venv/lib/python3.13/site-packages/mcp/client/streamable_http.py#134\134]8;;\

                    INFO     Negotiated protocol version: 2025-06-18                         ]8;id=177856;file:///Users/sbattoo/Src/AIActivationDay/samples/.venv/lib/python3.13/site-packages/mcp/client/streamable_http.py\streamable_http.py]8;;\:]8;id=172428;file:///Users/sbattoo/Src/AIActivationDay/samples/.venv/lib/python3.13/site-packages/mcp/client/streamable_http.py#146\146]8;;\

                    INFO     HTTP Request: POST http://localhost:8000/mcp "HTTP/1.1 202 Accepted"   ]8;id=724839;file:///Users/sbattoo/Src/AIActivationDay/samples/.venv/lib/python3.13/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=38894;file:///Users/sbattoo/Src/AIActivationDay/samples/.venv/lib/python3.13/site-packages/httpx/_client.py#1740\1740]8;;\

...

                    INFO     HTTP Request: POST http://localhost:8000/mcp "HTTP/1.1 200 OK"         ]8;id=973808;file:///Users/sbattoo/Src/AIActivationDay/samples/.venv/lib/python3.13/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=346247;file:///Users/sbattoo/Src/AIActivationDay/samples/.venv/lib/python3.13/site-packages/httpx/_client.py#1740\1740]8;;\

                    INFO     Processing request of type ListToolsRequest                              ]8;id=535206;file:///Users/sbattoo/Src/AIActivationDay/samples/.venv/lib/python3.13/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=958592;file:///Users/sbattoo/Src/AIActivationDay/samples/.venv/lib/python3.13/site-packages/mcp/server/lowlevel/server.py#623\623]8;;\

Calculation result: 30


                    INFO     Terminating session: 7e8a1c1874f24d72b2c6ba57ba9dc570           ]8;id=513105;file:///Users/sbattoo/Src/AIActivationDay/samples/.venv/lib/python3.13/site-packages/mcp/server/streamable_http.py\streamable_http.py]8;;\:]8;id=183629;file:///Users/sbattoo/Src/AIActivationDay/samples/.venv/lib/python3.13/site-packages/mcp/server/streamable_http.py#630\630]8;;\

                    INFO     HTTP Request: DELETE http://localhost:8000/mcp "HTTP/1.1 200 OK"       ]8;id=723048;file:///Users/sbattoo/Src/AIActivationDay/samples/.venv/lib/python3.13/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=849458;file:///Users/sbattoo/Src/AIActivationDay/samples/.venv/lib/python3.13/site-packages/httpx/_client.py#1740\1740]8;;\

You can optionally also provide `read_timeout_seconds` while calling an MCP server tool to avoid it running for too long

In [ ]:
with streamable_http_mcp_client:
    try:
        result = streamable_http_mcp_client.call_tool_sync(
            tool_use_id="tool-123",
            name="long_running_tool",
            arguments={"name": "Amazon"},
            read_timeout_seconds=timedelta(seconds=30),
        )

        if result["status"] == "error":
            print(f"Tool execution failed: {result['content'][0]['text']}")
        else:
            print(f"Tool execution succeeded: {result['content'][0]['text']}")
    except Exception as e:
        print(f"Tool call timed out or failed: {str(e)}")

### Interacting with multiple MCP servers

With Strands Agents you can also interact with multiple MCP servers using the same agent and configure tools setups such as the max number of tools that can be used in parallel (`max_parallel_tools`). Let's create a new agent to showcase this configuration:

<div style="text-align:center">
    <img src="images/architecture_3.png" width="85%" />
</div>

In this agent, we will again use the AWS Documentation MCP server and we will also use the [AWS CDK MCP Server](https://awslabs.github.io/mcp/servers/cdk-mcp-server/) which helps with AWS Cloud Development Kit (CDK) best practices, infrastructure as code patterns and security compliance with CDK Nag.

First let's connect to the two MCP servers using the stdio transport

In [10]:
# Connect to an MCP server using stdio transport
aws_docs_mcp_client = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(
            command="uvx", args=["awslabs.aws-documentation-mcp-server@latest"]
        )
    )
)

# Connect to an MCP server using stdio transport
cdk_mcp_client = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(command="uvx", args=["awslabs.cdk-mcp-server@latest"])
    )
)

#### Create Agent with MCP servers

Next we will create the agent with the tools from both MCP servers

In [11]:
# Create an agent with MCP tools
with aws_docs_mcp_client, cdk_mcp_client:
    # Get the tools from the MCP server
    tools = aws_docs_mcp_client.list_tools_sync() + cdk_mcp_client.list_tools_sync()

    # Create an agent with these tools
    agent = Agent(
        model="global.anthropic.claude-sonnet-4-20250514-v1:0",
        tools=tools)

    response = agent(
        "What is Amazon Bedrock pricing model. Be concise. Also what are the best practices related to CDK?"
    )

[09/16/25 14:26:16] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=461533;file:///Users/sbattoo/Src/AIActivationDay/samples/.venv/lib/python3.13/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=99735;file:///Users/sbattoo/Src/AIActivationDay/samples/.venv/lib/python3.13/site-packages/botocore/credentials.py#1359\1359]8;;\

I'll help you with both questions. Let me search for Amazon Bedrock pricing information and get CDK best practices guidance.
Tool #1: search_documentation

Tool #2: CDKGeneralGuidance
Let me get the specific Bedrock pricing details from the documentation:
Tool #3: read_documentation
## Amazon Bedrock Pricing Model (Concise)

Amazon Bedrock uses a **pay-per-use model** based on:

1. **Token-based pricing**: Charged separately for input tokens and output tokens
2. **On-demand inference**: Pay as you go with no upfront costs
3. **Provisioned Throughput**: Optional capacity purchasing for consistent performance and potentially lower per-token costs

**Key points:**
- Different models have different pricing per token
- Input and output tokens are priced separately
- Provisioned Throughput offers committed capacity for high-volume use cases
- Additional charges apply for Guardrails evaluation when used

## CDK Best Practices Summary

### Core Development Principles
- **Always use `cdk synth`

### Congratulations!

In this notebook you learned how to connect with MCP servers using Strands Agent and two MCP transport protocols: stdio and Streamable HTTP. You also learned how to connect multiple MCP servers to the same agent. Next, let's see how to use different models with your agent